In [ ]:
import pvl
import struct
import matplotlib.pyplot as plt
import numpy as np
import datetime
import os.path
import binascii


In [ ]:
mimap_file = 'path/to/input/file.IMG'
image_file = mimap_file

In [ ]:
header = pvl.load(mimap_file)
header

In [ ]:
with open(mimap_file, 'rb') as f:
    image_offset = ((header["^IMAGE"] -1) * (header["RECORD_BYTES"])) -1
    f.seek(image_offset)
    b_image_data = f.read()

print(b_image_data[0:20], image_offset)


In [ ]:
n_lines = 4
n_samples = 5
line_length = header['IMAGE']['LINE_SAMPLES'] * (header['IMAGE']['SAMPLE_BITS']//8)

In [ ]:
image_data = []
line_offset = 100
sample_offset = 100

for j in range(line_offset, n_lines * header['IMAGE']['BANDS'] + line_offset + 1):
    print(j*line_length,(j+1)*line_length)
    image_sample = np.frombuffer(b_image_data[j*line_length:(j+1)*line_length], dtype=np.int16, count=int(line_length/2))
    image_data.append(image_sample)
image_data = np.array(image_data)
image_data = image_data[:, sample_offset:sample_offset + n_samples]
print(image_data.shape)

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(image_data,cmap="gray")#[0:20,0:20])

In [ ]:
print(image_data[0][0])


class RealIsisCubeLabelEncoder(pvl.encoder.ISISEncoder):
    def encode_time(self, value):
        if value.microsecond:
            second = u'%02d.%06d' % (value.second, value.microsecond)
        else:
            second = u'%02d' % value.second
            time = u'%02d:%02d:%s' % (value.hour, value.minute, second)
        return time



In [ ]:
import math

image_fn, image_ext = os.path.splitext(image_file)
crop = '_cropped'
mini_image_fn = image_fn + crop + image_ext
print(mini_image_fn)
mini_image_bn = os.path.basename(mini_image_fn)
grammar = pvl.grammar.ISISGrammar()
grammar.comments+=(("#", "\n"), )
encoder = RealIsisCubeLabelEncoder()
# Overwrite the number of lines in the label
header['IMAGE']['LINES'] = n_lines + 1
header['IMAGE']['LINE_SAMPLES'] = n_samples
actual_label_size = len(pvl.dumps(header, encoder=encoder, grammar=grammar)) / header["RECORD_BYTES"]
print(actual_label_size)
header['^IMAGE'] = math.ceil(actual_label_size) + 1
print(int(header["RECORD_BYTES"] * (math.ceil(actual_label_size) - actual_label_size)))
# - 1 from 0 based and another - 1 from the new line written before the padding
padding = ['0' for i in range(0, (int(header["RECORD_BYTES"] * (math.ceil(actual_label_size) - actual_label_size))) - 1)]
padding = ''.join(padding)
padding = padding.encode('utf8')
print(padding)

In [ ]:
label_fn, label_ext = os.path.splitext(mimap_file)
out_label = label_fn + crop + label_ext

pvl.dump(header, out_label, encoder=encoder, grammar=grammar)


In [ ]:
with open(mini_image_fn, 'ab+') as f:
    b_reduced_image_data = image_data.tobytes()
    f.write(b'\n')
    f.seek(0, 2)
    f.write(padding)
    f.write(b_reduced_image_data)
print(len(b_reduced_image_data),len(padding))

In [ ]:
pvl.load(mini_image_fn)

In [ ]:
with open(mini_image_fn, 'rb') as f:
  image_offset = ((header["^IMAGE"] -1) * header["RECORD_BYTES"])
  f.seek(image_offset)
  b_image_data = f.read()

In [ ]:
print(image_offset,b_image_data,len(b_image_data))
image_data = np.frombuffer(b_image_data, dtype=np.int16).reshape((n_lines + 1, n_samples))
plt.figure(0, figsize=(20, 20))
plt.imshow(image_data)
plt.show()